# Imports & dependencies

In [1]:
# Reading all_grpahs gives the same results in the adjacency matrices of the taxi
# what is meant by trans_tt, pcc_tt, and dis_tt????
# Those are taxt-taxt adjacencey matrices, but what do they mean exactly?

In [2]:
import torch
import numpy as np
import h5py
f1 = h5py.File('taxi_data.h5', 'r')
print(list(f1.keys()))

['taxi_drop', 'taxi_pick']


In [3]:
with h5py.File('taxi_data.h5', 'r') as hf:
    data_d = hf['taxi_pick'][:]
    data_s = hf['taxi_drop'][:]
    
data_d=np.array(data_d, dtype=int) 
data_s=np.array(data_s, dtype=int) 

In [4]:
print(data_d.shape)
print(data_s.shape)

(4368, 266)
(4368, 266)


In [5]:
print(4368/24)
print(24*9)

182.0
216


In [6]:
# extract training and testing arrays

In [7]:
import numpy as np
add_indices = np.arange(4368/2)
chunks = np.array(np.array_split(add_indices, 4368/12/2), dtype=int)
print(chunks[0].shape)

chunks2=chunks+4368/2
chunks2=np.array(chunks2, dtype=int)
print(type(chunks2))
# print(newarr[0])
# print(newarr[1]) 
# print(newarr[2])

(12,)
<class 'numpy.ndarray'>


In [8]:
X_train=np.zeros((266,24,180))
X_test=np.zeros((266,24,180))
for jj in np.arange(180):
    for i in np.arange(266):
        dem_i=data_d[:,i]
        sup_i=data_s[:,i]
        X_train[i,:, jj]=np.concatenate((dem_i[chunks[jj]], sup_i[chunks[jj]]))
        X_test[i,:, jj]=np.concatenate((dem_i[chunks2[jj]], sup_i[chunks2[jj]]))

#         print(dem_i.shape)
#     print(i)

In [9]:
# X_train[node number, 24 featues (dem+sup), batch index out of 180]
print(X_train.shape)
print(X_test.shape)

(266, 24, 180)
(266, 24, 180)


In [10]:
import pandas as pd
import numpy as np
a=pd.read_pickle('data/adj_data/adj_mx_taxi.pkl')
adj=np.array(a)
# adj2=adj[2]
adj_distance=adj[2]

/home/lambda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


In [11]:
X_train_new=np.zeros((266,24*9,180))
X_test_new=np.zeros((266,24*9,180))

In [12]:
for zz in np.arange(266):
    arr=adj_distance[zz,:]
    vicinity_nodes=(-arr).argsort()[:9]
#     print(vicinity_nodes)
    for kkk in np.arange(180):
        X_train_new[zz, :, kkk]=np.matrix.flatten(X_train[vicinity_nodes, :,kkk])
        X_test_new[zz, :, kkk]=np.matrix.flatten(X_test[vicinity_nodes, :,kkk])
#     X_test_new[zz, :, 0]=X_test[vicinity_nodes, :,0]

In [13]:
print(X_train_new.shape)
print(X_test_new.shape)

(266, 216, 180)
(266, 216, 180)


In [14]:
9*24

216

In [16]:
X_train_new=torch.from_numpy(X_train_new)
X_test_new=torch.from_numpy(X_test_new)
torch.save(X_train_new, 'X_train_new.pt')
torch.save(X_test_new, 'X_test_new.pt')